<a href="https://colab.research.google.com/github/kislay960/Emojify-it/blob/main/Emoji_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install the emoji package to display different emojis
!pip install emoji

In [ ]:
import emoji

In [ ]:
#creating a dictionary for the emojis we will use
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [ ]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [ ]:
#importing libraries required
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from keras.utils import np_utils

import matplotlib.pyplot as plt

In [ ]:
#loading the dataset
train = pd.read_csv('https://raw.githubusercontent.com/kislay960/Data-Science-Practice/main/train_emoji.csv',header=None)
test = pd.read_csv('https://raw.githubusercontent.com/kislay960/Data-Science-Practice/main/test_emoji.csv',header=None)

In [ ]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [ ]:
data = train.values
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [ ]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(132,)
(56,)
(132,)
(56,)


In [ ]:
# Converting labels into categorical form
Y_train = np_utils.to_categorical(Y_train)

In [ ]:
# Creating  embeddings dictionary with key = word and value = list of words in glove vector
embeddings_index = {}

f = open('/content/drive/MyDrive/glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
X_train[1]

'I am proud of your achievements'

In [ ]:
# Filling the embedding matrix
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings_index[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [ ]:
X_train[1]

'I am proud of your achievements'

In [ ]:
embedding_matrix_train = getOutputEmbeddings(X_train)
embedding_matrix_test = getOutputEmbeddings(X_test)

AttributeError: ignored

In [ ]:
print (embedding_matrix_train.shape, embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


In [ ]:
# A simple RNN and LSTM network to classify the emoji class from an input Sentence

model = Sequential()
model.add(SimpleRNN(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 10, 64)            7360      
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 40,709
Trainable params: 40,709
Non-trainable params: 0
____________________________________________________

In [ ]:
# Setting Loss and Optimiser
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Training the model and Setting hyperparameters
hist = model.fit(embedding_matrix_train,Y_train,epochs=100, batch_size=64,shuffle=True)

Epoch 1/100
3/3 [==============================] - 0s 9ms/step - loss: 1.6332 - accuracy: 0.2045
Epoch 2/100
3/3 [==============================] - 0s 8ms/step - loss: 1.5920 - accuracy: 0.2197
Epoch 3/100
3/3 [==============================] - 0s 8ms/step - loss: 1.5017 - accuracy: 0.3712
Epoch 4/100
3/3 [==============================] - 0s 8ms/step - loss: 1.4870 - accuracy: 0.3106
Epoch 5/100
3/3 [==============================] - 0s 10ms/step - loss: 1.4681 - accuracy: 0.4015
Epoch 6/100
3/3 [==============================] - 0s 8ms/step - loss: 1.4277 - accuracy: 0.3712
Epoch 7/100
3/3 [==============================] - 0s 8ms/step - loss: 1.4152 - accuracy: 0.4242
Epoch 8/100
3/3 [==============================] - 0s 8ms/step - loss: 1.3459 - accuracy: 0.4621
Epoch 9/100
3/3 [==============================] - 0s 8ms/step - loss: 1.2872 - accuracy: 0.5455
Epoch 10/100
3/3 [==============================] - 0s 8ms/step - loss: 1.2681 - accuracy: 0.5606
Epoch 11/100
3/3 [==========

In [ ]:
#predicting values for test data
pred = model.predict_classes(embedding_matrix_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
pred

array([4, 3, 3, 0, 2, 2, 3, 2, 4, 2, 1, 2, 0, 3, 1, 0, 3, 2, 3, 4, 0, 0,
       4, 0, 3, 3, 2, 0, 3, 2, 0, 1, 0, 2, 0, 1, 2, 3, 4, 2, 2, 0, 0, 1,
       2, 1, 2, 2, 0, 1, 1, 3, 3, 2, 2, 2])

In [ ]:
for i in range(30):
    print(' '.join(X_test[i]))
    print('Predicted emoji : ' + emoji.emojize(emoji_dictionary[str(pred[i])]) + '\n')

I want to eat
Predicted emoji : 🍴

he did not answer
Predicted emoji : 😓

he got a raise
Predicted emoji : 😓

she got me a present
Predicted emoji : ❤️

ha ha ha it was so funny
Predicted emoji : 😁

he is a good friend
Predicted emoji : 😁

I am upset
Predicted emoji : 😓

We had such a lovely dinner tonight
Predicted emoji : 😁

where is the food
Predicted emoji : 🍴

Stop making this joke ha ha ha
Predicted emoji : 😁

where is the ball
Predicted emoji : ⚾

work is hard
Predicted emoji : 😁

This girl is messing with me
Predicted emoji : ❤️

are you serious ha ha
Predicted emoji : 😓

Let us go play baseball
Predicted emoji : ⚾

This stupid grader is not working
Predicted emoji : ❤️

work is horrible
Predicted emoji : 😓

Congratulation for having a baby
Predicted emoji : 😁

stop messing around
Predicted emoji : 😓

any suggestions for dinner
Predicted emoji : 🍴

I love taking breaks
Predicted emoji : ❤️

you brighten my day
Predicted emoji : ❤️

I boiled rice
Predicted emoji : 🍴

she is a bu

In [ ]:
with open("model.json", "w") as file:
    file.write(model.to_json())
model.save_weights("model.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
with open("model.json", "r") as file:
    model = model_from_json(file.read())
model.load_weights("model.h5")

In [ ]:
test_str = "Hello how are you"
X = pd.Series([test_str])

In [ ]:
emb_X = getOutputEmbeddings(X)

NameError: ignored